# This is for PASCAL-VOC XML to YOLOV8 txt files.

In [4]:
import os
import sys
from xml.dom.minidom import parse

def cord_converter(size, box):
    """
    Convert xml annotation to darknet format coordinates
    :param size: [w,h]
    :param box: anchor box coordinates [upper-left x, upper-left y, lower-right x, lower-right y]
    :return: converted [x,y,w,h]
    """
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    dw = 1. / int(size[0])
    dh = 1. / int(size[1])

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

def save_file(output_folder, img_jpg_file_name, size, img_box):
    save_file_name = os.path.join(output_folder, img_jpg_file_name + '.txt')
    with open(save_file_name, "a+") as file_path:
        for box in img_box:                  
            cls_num = 0
            new_box = cord_converter(size, box[1:]) # convert box coord into YOLO x,y,w,h
            file_path.write(f"{cls_num} {' '.join(map(str, new_box))}\n")

def process_folder(input_folder, output_folder):
    for filename in os.listdir(input_folder):
        if filename.endswith('.xml'):
            img_xml_file = os.path.splitext(filename)[0]
            get_xml_data(input_folder, img_xml_file, output_folder)

def get_xml_data(file_path, img_xml_file, output_folder):
    img_path = os.path.join(file_path, img_xml_file + '.xml')

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = int(img_size.getElementsByTagName("width")[0].childNodes[0].data)
    img_h = int(img_size.getElementsByTagName("height")[0].childNodes[0].data)
    img_box = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)
        img_box.append([cls_name, x1, y1, x2, y2])
    save_file(output_folder, img_xml_file, [img_w, img_h], img_box)

## for train

In [5]:
# Replace 'input_folder' with the path to your folder containing XML files
input_folder = 'C:/Users/aniru/Desktop/fresh/Full_Indian_NPs/Dataset/train/annots'
# Replace 'output_folder' with the path to your folder where you want to save the text files
output_folder = 'C:/Users/aniru/Desktop/fresh/Full_Indian_NPs/Dataset/train/labels'
process_folder(input_folder, output_folder)

## for val

In [6]:
# Replace 'input_folder' with the path to your folder containing XML files
input_folder = 'C:/Users/aniru/Desktop/fresh/Full_Indian_NPs/Dataset/val/annots'
# Replace 'output_folder' with the path to your folder where you want to save the text files
output_folder = 'C:/Users/aniru/Desktop/fresh/Full_Indian_NPs/Dataset/val/labels'
process_folder(input_folder, output_folder)